## Best Console Analysis

Using the same data set of video game sales, I want to determine "What is the best video game console to invest in based on the number of game copies sold globally over the past 10 years

This will include: 

-Filtering for just the past 10 years

-Grouping by console

-Summing all global sales by console

In [19]:
#Setting up our imports
import pandas as pd
import matplotlib.pyplot as plt

First, we'll import our .csv file so we can begin our data cleanup!  Instead of a standard index column beginning at 0, let's use the Rank column for indexing our rows!

We can specify an index column using the index_col parameter!

In [20]:
vg_sales = pd.read_csv("vgsales.csv", index_col="Rank")
vg_sales.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Looking good!  Next, let's do some initial cleanup of this dataset

First, we'll look at vg_sales.info() to determine any null/NaN values

In [21]:
vg_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16598 entries, 1 to 16600
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16598 non-null  object 
 1   Platform      16598 non-null  object 
 2   Year          16327 non-null  float64
 3   Genre         16598 non-null  object 
 4   Publisher     16540 non-null  object 
 5   NA_Sales      16598 non-null  float64
 6   EU_Sales      16598 non-null  float64
 7   JP_Sales      16598 non-null  float64
 8   Other_Sales   16598 non-null  float64
 9   Global_Sales  16598 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.4+ MB


Note that there are 16598 entries, but Year and Publisher both contain null values!

For our analysis, we need to ensure Year does NOT have a null value!

We can utilize dropna() to remove those rows from our dataset

As the dataset is quite large, this should not impact overall data analysis


In [22]:
vg_sales.dropna(subset = ["Year"],inplace=True)
vg_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16327 entries, 1 to 16600
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16327 non-null  object 
 1   Platform      16327 non-null  object 
 2   Year          16327 non-null  float64
 3   Genre         16327 non-null  object 
 4   Publisher     16291 non-null  object 
 5   NA_Sales      16327 non-null  float64
 6   EU_Sales      16327 non-null  float64
 7   JP_Sales      16327 non-null  float64
 8   Other_Sales   16327 non-null  float64
 9   Global_Sales  16327 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.4+ MB


Quite a few rows have been removed, but we still have over 16000 records in our database, now with no null values!

Next, let's see if any of these can be changed to Categories to reduce data set size.

First, we'll use .nunique() to determine how many unique values each column has

In [23]:
vg_sales.nunique()

Name            11360
Platform           31
Year               39
Genre              12
Publisher         576
NA_Sales          408
EU_Sales          305
JP_Sales          244
Other_Sales       157
Global_Sales      621
dtype: int64

There are very few unique values for Platform, Year, and Genre!

We'll hold off on converting Year (up next we'll convert it from a float to an integer!)

For Platform and Genre, let's convert to Categories, and see the new info for our dataframe

In [24]:
vg_sales["Platform"] = vg_sales["Platform"].astype('category')
vg_sales["Genre"] = vg_sales["Genre"].astype('category')
vg_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16327 entries, 1 to 16600
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Name          16327 non-null  object  
 1   Platform      16327 non-null  category
 2   Year          16327 non-null  float64 
 3   Genre         16327 non-null  category
 4   Publisher     16291 non-null  object  
 5   NA_Sales      16327 non-null  float64 
 6   EU_Sales      16327 non-null  float64 
 7   JP_Sales      16327 non-null  float64 
 8   Other_Sales   16327 non-null  float64 
 9   Global_Sales  16327 non-null  float64 
dtypes: category(2), float64(6), object(2)
memory usage: 1.2+ MB


Perfect, that brought our dataset size down to 1.2 MB!

Now, let's convert the float64 for Year into an integer, to save space!

In [26]:
vg_sales["Year"].astype(int)
vg_sales["Year"] = vg_sales["Year"].astype(int)
vg_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16327 entries, 1 to 16600
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Name          16327 non-null  object  
 1   Platform      16327 non-null  category
 2   Year          16327 non-null  int32   
 3   Genre         16327 non-null  category
 4   Publisher     16291 non-null  object  
 5   NA_Sales      16327 non-null  float64 
 6   EU_Sales      16327 non-null  float64 
 7   JP_Sales      16327 non-null  float64 
 8   Other_Sales   16327 non-null  float64 
 9   Global_Sales  16327 non-null  float64 
dtypes: category(2), float64(5), int32(1), object(2)
memory usage: 1.1+ MB


Perfect, now let's start analyzing!  First, let's sort our table by year, and see a preview of the top 5 & bottom 5 rows

In [32]:
vg_sales.sort_values(by="Year", ascending=False ,inplace=True)
vg_sales

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
5959,Imagine: Makeup Artist,DS,2020,Simulation,Ubisoft,0.27,0.00,0.00,0.02,0.29
16441,Brothers Conflict: Precious Baby,PSV,2017,Action,Idea Factory,0.00,0.00,0.01,0.00,0.01
16244,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017,Role-Playing,Sega,0.00,0.00,0.01,0.00,0.01
14393,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017,Role-Playing,Sega,0.00,0.00,0.03,0.00,0.03
9540,Dragon Quest Heroes II: Twin Kings and the Pro...,PS3,2016,Action,Square Enix,0.00,0.00,0.13,0.00,0.13
...,...,...,...,...,...,...,...,...,...,...
2671,Boxing,2600,1980,Fighting,Activision,0.72,0.04,0.00,0.01,0.77
545,Missile Command,2600,1980,Shooter,Atari,2.56,0.17,0.00,0.03,2.76
259,Asteroids,2600,1980,Shooter,Atari,4.00,0.26,0.00,0.05,4.31


The Year value starts at 1980 and goes up to 2020!  For the purpose of this analysis, we are only looking for games made in 2021 or after.

So, let's start querying our dataset for the relevant information!

In [33]:
last_decade = vg_sales["Year"] >= 2012
last_decade

Rank
5959      True
16441     True
16244     True
14393     True
9540      True
         ...  
2671     False
545      False
259      False
4027     False
6898     False
Name: Year, Length: 16327, dtype: bool

Note that this is simply a boolean series - True if the value matches the condition, False otherwise.

To use this to filter our dataset, we can do the following:

In [34]:
vg_last_decade = vg_sales[last_decade]
vg_last_decade

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
5959,Imagine: Makeup Artist,DS,2020,Simulation,Ubisoft,0.27,0.00,0.00,0.02,0.29
16441,Brothers Conflict: Precious Baby,PSV,2017,Action,Idea Factory,0.00,0.00,0.01,0.00,0.01
16244,Phantasy Star Online 2 Episode 4: Deluxe Package,PSV,2017,Role-Playing,Sega,0.00,0.00,0.01,0.00,0.01
14393,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017,Role-Playing,Sega,0.00,0.00,0.03,0.00,0.03
9540,Dragon Quest Heroes II: Twin Kings and the Pro...,PS3,2016,Action,Square Enix,0.00,0.00,0.13,0.00,0.13
...,...,...,...,...,...,...,...,...,...,...
14519,Doom 3 BFG Edition,PC,2012,Shooter,Bethesda Softworks,0.00,0.02,0.00,0.01,0.03
16168,Ouka Sengoku Portable,PSP,2012,Adventure,Alchemist,0.00,0.00,0.01,0.00,0.01
624,NBA 2K13,PS3,2012,Sports,Take-Two Interactive,1.72,0.44,0.05,0.27,2.48


Looking good!  Next, we need to group the data together for the same Platform values

To do this we can utilize .groupby()

From there, we can chain the column we want to sum (Global Sales), and sort in descending order

In [41]:
vg_last_decade.groupby("Platform")
vg_last_decade.groupby("Platform")["Global_Sales"].sum().sort_values(ascending=False).head()

Platform
PS3     298.65
PS4     278.10
X360    240.79
3DS     183.75
XOne    141.06
Name: Global_Sales, dtype: float64

Based on our analysis, it seems that the PS3 is the best console to invest in, based on global sales from the past 10 years!